# First Challenge TheBigBatchTheory
Notebook of the first challenge for the AN2DL course 2025-2026

Components:
- Benedetta Mussini
- Andrea Rossi
- Fabio Rossi
- Francesco Sarra

### Setup and Import

In [ ]:
# Set seed for reproducibility
SEED = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(SEED)
random.seed(SEED)

# Import PyTorch
import torch
torch.manual_seed(SEED)
from torch import nn
# from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
logs_dir = "tensorboard"
!pkill -f tensorboard
%load_ext tensorboard
!mkdir -p models

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device("cpu")

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

# Import other libraries
import copy
import shutil
from itertools import product
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

# Define dataset names and their corresponding Google Drive URLs
dataset_names = [
    "pirate_pain_train.csv",
    "pirate_pain_train_labels.csv",
    "pirate_pain_test.csv"
]

dataset_URLs = {
    dataset_names[0]: "1hgDec_3kcDrM6voSSvzhxW4oBlot46fQ",
    dataset_names[1]: "1TgebFnodrCFVme4Rha3kZVgFfW114qb2",
    dataset_names[2]: "12F3ckAhM39q8D3S5zBDU1Smb1FK8eygr"
}

# Loop through each dataset and download if not already present
for name in dataset_names:
    if not os.path.exists(name):
        print(f"Downloading {name}...")
        !gdown -q {dataset_URLs[name]} -O {name}
        print(f"{name} downloaded!")
    else:
        print(f"{name} already downloaded. Using cached data.")

### Data Profiling

In [ ]:
import sys
!{sys.executable} -m pip install -U ydata-profiling[notebook]
!pip install jupyter-contrib-nbextensions

!jupyter nbextension enable --py widgetsnbextension
!pip install dataprofiler

from ydata_profiling import ProfileReport
import json

df = pd.read_csv('pirate_pain_train.csv')
report = ProfileReport(df, title="Pandas Profiling Report")
report.to_file("report.html")
report

#### Considerations

After performing an extensive exploration of the dataset, we applied several preprocessing steps to reduce noise, handle redundancy, address multicollinearity, and ensure that the downstream neural models would operate on a clean and informative feature space.

We first inspected the three pirate-related features (n_legs, n_hands, n_eyes), which were stored as strings. These columns were mapped into numeric values using a consistent dictionary, and we verified that the same transformation was applied to the test set to avoid unseen categories. A subsequent analysis revealed that these three variables always carried identical information; therefore, we merged them into a single synthetic feature called pirate, dropping the original columns to remove perfect redundancy.

Given the strong correlations observed across several groups of joint features, we applied PCA to selected subsets (joint_00–05, joint_06–07, joint_08–09, joint_10–11, and joint_26–29). This dimensionality reduction step allowed us to compress highly correlated signals while retaining most of the original variance. All scaling operations were performed with a StandardScaler fitted only on the training data to prevent data leakage. After PCA, the original joint columns used in each block were removed from all datasets.

Before moving into model training, we additionally filtered out joint features with negligible predictive strength. To achieve this, we one-hot encoded the target labels and computed correlations between each joint and the three resulting indicator columns. We performed this analysis specifically on joints that were mostly zero-valued (almost always inactive) with only occasional spikes, since these sparse features tend to produce unstable or misleading correlations if evaluated over the full dataset. For each of these joints, correlations were calculated only on non-zero samples. If the absolute correlation consistently remained within the interval [−0.3,0.3], the feature was deemed uninformative and was removed from the training, validation and test sets.

The survey features (pain_survey_1–4) were categorical rather than ordinal, and therefore we applied One-Hot Encoding to all of them. This expanded the feature space but ensured that their categorical semantics were handled correctly without imposing artificial order.

To avoid leakage across individuals, the train/validation split was performed at the level of unique sample_index, ensuring temporal sequences from the same subject never appeared in both sets.

Normalization was applied to all PCA-derived and continuous features using min–max scaling. Importantly, the scaling parameters were computed only from the training set, ensuring consistent but leakage-free transformations on validation and test sets.

Finally, since the dataset is imbalanced we computed log-scaled class weights to stabilise learning and avoid dominance by majority classes. These weights were normalised and passed to the loss function.

## Data Exploration and Preprocessing

In [ ]:
# Loading dataset
df = pd.read_csv('pirate_pain_train.csv')
df_test = pd.read_csv('pirate_pain_test.csv')
df.head()

In [ ]:
# Map strings in n_legs/n_hands/n_eyes to numbers
num_map = {'one+hook_hand':1, 'one+eye_patch':1, 'two':2, 'one+peg_leg':1}
for col in ['n_legs','n_hands','n_eyes']:
    if col in df.columns and df[col].dtype == 'O':
        df[col] = df[col].map(num_map).astype('float32')
        df_test[col] = df_test[col].map(num_map).astype('float32')

# Substitute 'n_eyes', 'n_legs', 'n_hands' columns with 'pirate' column if all three columns have the same values
if (df['n_eyes'] - df['n_legs']).nunique() and (df['n_eyes'] - df['n_hands']).nunique() and (df['n_hands'] - df['n_legs']).nunique() :
  df.drop('n_eyes', axis=1, inplace=True)
  df.drop('n_hands', axis=1, inplace=True)
  df['pirate']=df['n_legs']
  df.drop('n_legs', axis=1, inplace=True)

if (df_test['n_eyes'] - df_test['n_legs']).nunique() and (df_test['n_eyes'] - df_test['n_hands']).nunique() and (df_test['n_hands'] - df_test['n_legs']).nunique() :
  df_test.drop('n_eyes', axis=1, inplace=True)
  df_test.drop('n_hands', axis=1, inplace=True)
  df_test['pirate']=df_test['n_legs']
  df_test.drop('n_legs', axis=1, inplace=True)

In [ ]:
# PCA on high correlations columns
def apply_pca(df, df_test, columns_to_pca, n_components, new_columns_name):

    #Select column to perform pca on
    df_subset = df[columns_to_pca]

    #Data scaling
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df_subset)
    scaled_data_test = scaler.transform(df_test[columns_to_pca])

    #Apply PCA
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(scaled_data)
    principal_components_test = pca.transform(scaled_data_test)

    #Analyze results
    print(f"Numero di componenti selezionate: {pca.n_components_}")
    print(f"Varianza spiegata da ciascuna componente: {pca.explained_variance_ratio_}")

    #Create a new DataFrame with the principal components
    pc_columns = new_columns_name
    df_pca = pd.DataFrame(data=principal_components, columns=pc_columns)
    df_pca_test  = pd.DataFrame(data=principal_components_test,  columns=new_columns_name)

    # Merge new components to the original DataFrame removing the old ones.
    # Reset indexes
    df_reset = df.reset_index(drop=True)
    df_test_reset = df_test.reset_index(drop=True)
    df_pca_final = pd.concat([df_reset, df_pca], axis=1)
    df_test_final = pd.concat([df_test_reset, df_pca_test], axis=1)

    # Remove original columns
    df_pca_final = df_pca_final.drop(columns=columns_to_pca)
    df_test_final = df_test_final.drop(columns=columns_to_pca)

    print(f"PCA applied to {columns_to_pca}")

    return df_pca_final , df_test_final

df, df_test = apply_pca(df=df, df_test=df_test, columns_to_pca=['joint_00', 'joint_01', 'joint_02', 'joint_03', 'joint_04', 'joint_05'], n_components=2, new_columns_name=['joint_00-05_x', 'joint_00-05_y'])
df, df_test = apply_pca(df=df, df_test=df_test, columns_to_pca=['joint_06', 'joint_07'], n_components=1, new_columns_name=['joint_06-07'])
df, df_test = apply_pca(df=df, df_test=df_test, columns_to_pca=['joint_08', 'joint_09'], n_components=1, new_columns_name=['joint_08-09'])
df, df_test = apply_pca(df=df, df_test=df_test, columns_to_pca=['joint_10', 'joint_11'], n_components=1, new_columns_name=['joint_10-11'])
df, df_test = apply_pca(df=df, df_test=df_test, columns_to_pca=['joint_26', 'joint_27', 'joint_28', 'joint_29'], n_components=2, new_columns_name=['joint_26-29_x', 'joint_26-29_y'])

In [ ]:
# drop column joint_30 because constant
df = df.drop('joint_30', axis=1)
df_test = df_test.drop('joint_30', axis=1)

# One-Hot Encoding on 'label' column (to explore correlation)
y_train = pd.read_csv('pirate_pain_train_labels.csv')
label_col = [c for c in y_train.columns if c != 'sample_index'][0]
df = df.merge(y_train[['sample_index', label_col]], on='sample_index', how='left')
label_mapping = {
    'no_pain': 0,
    'low_pain': 1,
    'high_pain': 2,
}

df['label'] = df['label'].map(label_mapping)

# dataset to explore correlation
df_exploration = pd.get_dummies(df, columns=['label'], prefix='label')

In [ ]:
# Drop columns with low correlation with 'label'
# Define columns to be handled
joint_columns = ['joint_13', 'joint_14', 'joint_15', 'joint_16', 'joint_17', 'joint_18',
       'joint_19', 'joint_20', 'joint_21', 'joint_22', 'joint_23', 'joint_24',
       'joint_25']

# Define new OHE columns
ohe_label_columns = ['label_0', 'label_1', 'label_2']

# List to compare ohe with the current column
columns_to_correlate = []

print("--- Calcolo Correlazioni (filtrate) dopo OHE ---\n")

for j in joint_columns:
    df_filtered = df_exploration[df_exploration[j] != 0]

    # Select current column and the ohe columns
    columns_to_correlate = [j] + ohe_label_columns

    # Calculate correlation matrix on the column subset
    correlation_matrix = df_filtered[columns_to_correlate].corr()

    # Print correlation values

    print(f"--- Correlazioni per {j} (dove {j} != 0) ---")

    print(correlation_matrix[j].drop(j))
    print("\n")

    # Drop columns that have correlation lower than 0.3 (in absolute value) wit 'label'
    if (correlation_matrix[j].drop(j).min() > -0.3) and (correlation_matrix[j].drop(j).max() <0.3):
        print(f"Drop column {j}. Min = {correlation_matrix[j].min()}, max = {correlation_matrix[j].max()}")
        df_exploration.drop(j, axis=1, inplace=True)
        df.drop(j, axis=1, inplace=True)
        df_test.drop(j, axis=1, inplace=True)
    else:
      print(f"Min = {correlation_matrix[j].drop(j).min()}, max = {correlation_matrix[j].drop(j).max()}")


In [ ]:
# One-Hot Encoding on pain_survey
# Define columns to be encoded
survey_columns = ['pain_survey_1', 'pain_survey_2', 'pain_survey_3', 'pain_survey_4']

# Apply One-Hot Encoding on them
df_exploration = pd.get_dummies(df_exploration, columns=survey_columns)
df = pd.get_dummies(df, columns=survey_columns)
df_test = pd.get_dummies(df_test, columns=survey_columns)

In [ ]:
# This is the final dataset for training
df.head()

#### Split Train/Validation and normalization

In [ ]:
PERCENTAGE_VALIDATION = 0.1

# Obtain unique subject IDs
unique_samples = df['sample_index'].unique()
random.shuffle(unique_samples)

# Split IDs
n_train_subjects = int((1-PERCENTAGE_VALIDATION) * np.floor(len(unique_samples)))
train_subjects = unique_samples[:n_train_subjects]
val_subjects   = unique_samples[n_train_subjects:]

# Create training and validation subsets
df_train = df[df['sample_index'].isin(train_subjects)]
df_val   = df[df['sample_index'].isin(val_subjects)]

# Define the columns to be normalised
scale_columns = ['joint_12', 'pirate', 'joint_00-05_x',
       'joint_00-05_y', 'joint_06-07', 'joint_08-09', 'joint_10-11',
       'joint_26-29_x', 'joint_26-29_y']

# Calculate the minimum and maximum values from the training data only
mins = df_train[scale_columns].min()
maxs = df_train[scale_columns].max()

df_test_pre_normalization = df_test.copy()

# Apply normalisation to the specified columns in all datasets
for column in scale_columns:
    # Normalise the training set
    df_train[column] = (df_train[column] - mins[column]) / (maxs[column] - mins[column])

    # Normalise the validation set
    df_val[column] = (df_val[column] - mins[column]) / (maxs[column] - mins[column])

    # Normalise the test set
    df_test[column] = (df_test[column] - mins[column]) / (maxs[column] - mins[column])


In [ ]:
# Computation of class weights for loss function (because of unbalanced training dataset)
# Compute the counts of each class
counts = np.bincount(df['label'])/160

# Calculate weights using a logaritmich scale
# log1p è log(1+x), che gestisce conteggi molto piccoli in modo stabile
weights = 1.0 / np.log1p(counts)

# Normalize weights
weights = weights / np.sum(weights) * len(counts)

print(f"Counts: {counts}")
print(f"Weights: {weights}")

# Convert the weights to a tensor
weights_tensor = torch.tensor(weights, dtype=torch.float32).to(device)

In [ ]:
# Select columns for training (feature columns)
DATA_COLUMNS = df_train.columns.tolist()
for c in ['sample_index', 'time', 'label']:
  DATA_COLUMNS.remove(c)
DATA_COLUMNS

## Function Definition

In [ ]:
# Define a function to build sequences from the dataset
def build_sequences(df, window=200, stride=200):

    # Initialise lists to store sequences and their corresponding labels
    dataset = []
    labels = []

    # Iterate over unique IDs in the DataFrame
    for id in df['sample_index'].unique():
        # Extract sensor data for the current ID
        temp = df[df['sample_index'] == id][DATA_COLUMNS].values

        # Retrieve the activity label for the current ID
        label = df[df['sample_index'] == id]['label'].values[0]

        # Calculate padding length to ensure full windows
        padding_len = window - len(temp) % window

        # Create zero padding and concatenate with the data
        padding = np.zeros((padding_len, len(DATA_COLUMNS)), dtype='float32')
        temp = np.concatenate((temp, padding))

        # Build feature windows and associate them with labels
        idx = 0
        while idx + window <= len(temp):
            dataset.append(temp[idx:idx + window])
            labels.append(label)
            idx += stride

    # Convert lists to numpy arrays for further processing
    dataset = np.array(dataset)
    labels = np.array(labels)

    return dataset, labels

def make_loader(ds, batch_size, shuffle, drop_last):
    # Determine optimal number of worker processes for data loading
    cpu_cores = os.cpu_count() or 2
    num_workers = max(2, min(4, cpu_cores))

    # Create DataLoader with performance optimizations
    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
        pin_memory=True,  # Faster GPU transfer
        pin_memory_device="cuda" if torch.cuda.is_available() else "",
        prefetch_factor=4,  # Load 4 batches ahead
    )

class RecurrentClassifier(nn.Module):
    """
    Generic RNN classifier (RNN, LSTM, GRU).
    Uses the last hidden state for classification.
    """
    def __init__(
            self,
            input_size,
            hidden_size,
            num_layers,
            num_classes,
            rnn_type='GRU',        # 'RNN', 'LSTM', or 'GRU'
            bidirectional=False,
            dropout_rate=0.2
            ):
        super().__init__()

        self.rnn_type = rnn_type
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional
        self.dropout_rate = dropout_rate

        # Map string name to PyTorch RNN class
        rnn_map = {
            'RNN': nn.RNN,
            'LSTM': nn.LSTM,
            'GRU': nn.GRU
        }

        if rnn_type not in rnn_map:
            raise ValueError("rnn_type must be 'RNN', 'LSTM', or 'GRU'")

        rnn_module = rnn_map[rnn_type]

        # Dropout is only applied between layers (if num_layers > 1)
        dropout_val = dropout_rate if num_layers > 1 else 0

        # Create the recurrent layer
        self.rnn = rnn_module(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,       # Input shape: (batch, seq_len, features)
            bidirectional=bidirectional,
            dropout=dropout_val
        )

        # Calculate input size for the final classifier
        if self.bidirectional:
            classifier_input_size = hidden_size * 2 # Concat fwd + bwd
        else:
            classifier_input_size = hidden_size

        # Final classification layer
        self.classifier = nn.Linear(classifier_input_size, num_classes)

    def forward(self, x):
        """
        x shape: (batch_size, seq_length, input_size)
        """

        # rnn_out shape: (batch_size, seq_len, hidden_size * num_directions)
        rnn_out, hidden = self.rnn(x)

        # LSTM returns (h_n, c_n), we only need h_n
        if self.rnn_type == 'LSTM':
            hidden = hidden[0]

        # hidden shape: (num_layers * num_directions, batch_size, hidden_size)

        if self.bidirectional:
            # Reshape to (num_layers, 2, batch_size, hidden_size)
            hidden = hidden.view(self.num_layers, 2, -1, self.hidden_size)

            # Concat last fwd (hidden[-1, 0, ...]) and bwd (hidden[-1, 1, ...])
            # Final shape: (batch_size, hidden_size * 2)
            hidden_to_classify = torch.cat([hidden[-1, 0, :, :], hidden[-1, 1, :, :]], dim=1)
        else:
            # Take the last layer's hidden state
            # Final shape: (batch_size, hidden_size)
            hidden_to_classify = hidden[-1]

        # Get logits
        logits = self.classifier(hidden_to_classify)
        return logits

def train_one_epoch(model, train_loader, criterion, optimizer, scaler, device, l1_lambda=0, l2_lambda=0):
    """
    Perform one complete training epoch through the entire training dataset.

    Args:
        model (nn.Module): The neural network model to train
        train_loader (DataLoader): PyTorch DataLoader containing training data batches
        criterion (nn.Module): Loss function (e.g., CrossEntropyLoss, MSELoss)
        optimizer (torch.optim): Optimization algorithm (e.g., Adam, SGD)
        scaler (GradScaler): PyTorch's gradient scaler for mixed precision training
        device (torch.device): Computing device ('cuda' for GPU, 'cpu' for CPU)
        l1_lambda (float): Lambda for L1 regularization
        l2_lambda (float): Lambda for L2 regularization

    Returns:
        tuple: (average_loss, f1 score) - Training loss and f1 score for this epoch
    """
    model.train()  # Set model to training mode

    running_loss = 0.0
    all_predictions = []
    all_targets = []

    # Iterate through training batches
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # Move data to device (GPU/CPU)
        inputs, targets = inputs.to(device), targets.to(device)

        # Clear gradients from previous step
        optimizer.zero_grad(set_to_none=True)

        # Forward pass with mixed precision (if CUDA available)
        with torch.amp.autocast(device_type=device.type, enabled=(device.type == 'cuda')):
            logits = model(inputs)
            loss = criterion(logits, targets)

            # Add L1 regularization
            l1_norm = sum(p.abs().sum() for p in model.parameters())

            loss = loss + l1_lambda * l1_norm


        # Backward pass with gradient scaling
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Accumulate metrics
        running_loss += loss.item() * inputs.size(0)
        predictions = logits.argmax(dim=1)
        all_predictions.append(predictions.cpu().numpy())
        all_targets.append(targets.cpu().numpy())

    # Calculate epoch metrics
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_f1 = f1_score(
        np.concatenate(all_targets),
        np.concatenate(all_predictions),
        average='weighted'
    )

    return epoch_loss, epoch_f1

def validate_one_epoch(model, val_loader, criterion, device):
    """
    Perform one complete validation epoch through the entire validation dataset.

    Args:
        model (nn.Module): The neural network model to evaluate (must be in eval mode)
        val_loader (DataLoader): PyTorch DataLoader containing validation data batches
        criterion (nn.Module): Loss function used to calculate validation loss
        device (torch.device): Computing device ('cuda' for GPU, 'cpu' for CPU)

    Returns:
        tuple: (average_loss, accuracy) - Validation loss and accuracy for this epoch

    Note:
        This function automatically sets the model to evaluation mode and disables
        gradient computation for efficiency during validation.
    """
    model.eval()  # Set model to evaluation mode

    running_loss = 0.0
    all_predictions = []
    all_targets = []

    # Disable gradient computation for validation
    with torch.no_grad():
        for inputs, targets in val_loader:
            # Move data to device
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass with mixed precision (if CUDA available)
            with torch.amp.autocast(device_type=device.type, enabled=(device.type == 'cuda')):
                logits = model(inputs)
                loss = criterion(logits, targets)

            # Accumulate metrics
            running_loss += loss.item() * inputs.size(0)
            predictions = logits.argmax(dim=1)
            all_predictions.append(predictions.cpu().numpy())
            all_targets.append(targets.cpu().numpy())

    # Calculate epoch metrics
    epoch_loss = running_loss / len(val_loader.dataset)
    epoch_accuracy = f1_score(
        np.concatenate(all_targets),
        np.concatenate(all_predictions),
        average='weighted'
    )

    return epoch_loss, epoch_accuracy

def log_metrics_to_tensorboard(writer, epoch, train_loss, train_f1, val_loss, val_f1, model):
    """
    Log training metrics and model parameters to TensorBoard for visualization.

    Args:
        writer (SummaryWriter): TensorBoard SummaryWriter object for logging
        epoch (int): Current epoch number (used as x-axis in TensorBoard plots)
        train_loss (float): Training loss for this epoch
        train_f1 (float): Training f1 score for this epoch
        val_loss (float): Validation loss for this epoch
        val_f1 (float): Validation f1 score for this epoch
        model (nn.Module): The neural network model (for logging weights/gradients)

    Note:
        This function logs scalar metrics (loss/f1 score) and histograms of model
        parameters and gradients, which helps monitor training progress and detect
        issues like vanishing/exploding gradients.
    """
    # Log scalar metrics
    writer.add_scalar('Loss/Training', train_loss, epoch)
    writer.add_scalar('Loss/Validation', val_loss, epoch)
    writer.add_scalar('F1/Training', train_f1, epoch)
    writer.add_scalar('F1/Validation', val_f1, epoch)

    # Log model parameters and gradients
    for name, param in model.named_parameters():
        if param.requires_grad:
            # Check if the tensor is not empty before adding a histogram
            if param.numel() > 0:
                writer.add_histogram(f'{name}/weights', param.data, epoch)
            if param.grad is not None:
                # Check if the gradient tensor is not empty before adding a histogram
                if param.grad.numel() > 0:
                    if param.grad is not None and torch.isfinite(param.grad).all():
                        writer.add_histogram(f'{name}/gradients', param.grad.data, epoch)

def fit(model, train_loader, val_loader, epochs, criterion, optimizer, scaler, device,
        learning_rate, l1_lambda=0, l2_lambda=0, patience=0, evaluation_metric="val_f1",
        mode='max', restore_best_weights=True, writer=None, verbose=10, experiment_name=""):
    """
    Train the neural network model on the training data and validate on the validation data.

    Args:
        model (nn.Module): The neural network model to train
        train_loader (DataLoader): PyTorch DataLoader containing training data batches
        val_loader (DataLoader): PyTorch DataLoader containing validation data batches
        epochs (int): Number of training epochs
        criterion (nn.Module): Loss function (e.g., CrossEntropyLoss, MSELoss)
        optimizer (torch.optim): Optimization algorithm (e.g., Adam, SGD)
        scaler (GradScaler): PyTorch's gradient scaler for mixed precision training
        device (torch.device): Computing device ('cuda' for GPU, 'cpu' for CPU)
        l1_lambda (float): L1 regularization coefficient (default: 0)
        l2_lambda (float): L2 regularization coefficient (default: 0)
        patience (int): Number of epochs to wait for improvement before early stopping (default: 0)
        evaluation_metric (str): Metric to monitor for early stopping (default: "val_f1")
        mode (str): 'max' for maximizing the metric, 'min' for minimizing (default: 'max')
        restore_best_weights (bool): Whether to restore model weights from best epoch (default: True)
        writer (SummaryWriter, optional): TensorBoard SummaryWriter object for logging (default: None)
        verbose (int, optional): Frequency of printing training progress (default: 10)
        experiment_name (str, optional): Experiment name for saving models (default: "")

    Returns:
        tuple: (model, training_history) - Trained model and metrics history
    """
    # Initialize metrics tracking
    training_history = {
        'train_loss': [], 'val_loss': [],
        'train_f1': [], 'val_f1': []
    }

    # Configure early stopping if patience is set
    if patience > 0:
        patience_counter = 0
        best_metric = float('-inf') if mode == 'max' else float('inf')
        best_epoch = 0

    # Main training loop: iterate through epochs
    for epoch in range(1, epochs + 1):

        # Forward pass through training data, compute gradients, update weights
        train_loss, train_f1 = train_one_epoch(
            model, train_loader, criterion, optimizer, scaler, device, l1_lambda, l2_lambda
        )

        # Evaluate model on validation data without updating weights
        val_loss, val_f1 = validate_one_epoch(
            model, val_loader, criterion, device
        )

        # Store metrics for plotting and analysis
        training_history['train_loss'].append(train_loss)
        training_history['val_loss'].append(val_loss)
        training_history['train_f1'].append(train_f1)
        training_history['val_f1'].append(val_f1)

        # Write metrics to TensorBoard for visualization
        if writer is not None:
            log_metrics_to_tensorboard(
                writer, epoch, train_loss, train_f1, val_loss, val_f1, model
            )

        # Print progress every N epochs or on first epoch
        if verbose > 0:
            if epoch % verbose == 0 or epoch == 1:
                print(f"Epoch {epoch:3d}/{epochs} | "
                    f"Train: Loss={train_loss:.4f}, F1 Score={train_f1:.4f} | "
                    f"Val: Loss={val_loss:.4f}, F1 Score={val_f1:.4f}")

        # Early stopping logic: monitor metric and save best model
        if patience > 0:
            current_metric = training_history[evaluation_metric][-1]
            is_improvement = (current_metric > best_metric) if mode == 'max' else (current_metric < best_metric)

            if is_improvement:
                best_metric = current_metric
                best_epoch = epoch
                torch.save(model.state_dict(), "models/"+experiment_name+'_model.pt')
                torch.save(model, "models/"+experiment_name+'_model.pth')
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"Early stopping triggered after {epoch} epochs.")
                    break

    # Restore best model weights if early stopping was used
    if restore_best_weights and patience > 0:
        model.load_state_dict(torch.load("models/"+experiment_name+'_model.pt'))
        print(f"Best model restored from epoch {best_epoch} with {evaluation_metric} {best_metric:.4f}")

    # Save final model if no early stopping
    if patience == 0:
        torch.save(model.state_dict(), "models/"+experiment_name+'_model.pt')

    # Close TensorBoard writer
    if writer is not None:
        writer.close()

    return model, training_history

def build_sequences_test(df, window=200, stride=200):

    dataset = []
    user_ids = []

    for uid in df['sample_index'].unique():
        temp = df[df['sample_index'] == uid][DATA_COLUMNS].values
        padding_len = window - len(temp) % window
        if padding_len != window:
            padding = np.zeros((padding_len, len(DATA_COLUMNS)), dtype='float32')
            temp = np.concatenate((temp, padding))

        idx = 0
        while idx + window <= len(temp):
            dataset.append(temp[idx:idx + window])
            user_ids.append(uid)
            idx += stride

    dataset = np.array(dataset)
    user_ids = np.array(user_ids)

    return dataset, user_ids

In [ ]:
def run_model(rnn_type, bidirectional, window_size, stride, batch_size, experiment_name, learning_rate=1e-3, epochs=500, patience=50, dropout_rate=0.2, l1_lambda=0, l2_lambda=0, hidden_layers=2, hidden_size=128, criterion=nn.CrossEntropyLoss(weight=weights_tensor, label_smoothing=0.1), verbose=0):
  '''
  Runs the model with the given parameters.
  Requires rnn_type, bidirectional, window_size, stride, batch_size and experiment_name the rest is optional
  '''
  print(f"Training {experiment_name}")

  # Generate sequences and labels for the training set
  X_train, y_train = build_sequences(df_train, window_size, stride)
  X_train = np.array(X_train, dtype=np.float32)

  # Generate sequences and labels for the validation set
  X_val, y_val = build_sequences(df_val, window_size, stride)
  X_val = np.array(X_val, dtype=np.float32)

  # Define the input shape based on the training data
  input_shape = X_train.shape[1:]

  # Define the number of classes based on the categorical labels
  num_classes = len(np.unique(y_train))

  # Convert numpy arrays to PyTorch datasets (pairs features with labels)
  train_ds = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
  val_ds   = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))

  # Create data loaders with different settings for each phase
  train_loader = make_loader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
  val_loader   = make_loader(val_ds, batch_size=batch_size, shuffle=False, drop_last=False)

  # Create model and display architecture with parameter count
  rnn_model = RecurrentClassifier(
      input_size=input_shape[-1], # Pass the number of features
      hidden_size=hidden_size,
      num_layers=hidden_layers,
      num_classes=num_classes,
      dropout_rate=dropout_rate,
      bidirectional=bidirectional,
      rnn_type=rnn_type
      ).to(device)

  # Define optimizer with L2 regularization
  optimizer = torch.optim.AdamW(rnn_model.parameters(), lr=learning_rate, weight_decay=l2_lambda)

  # Enable mixed precision training for GPU acceleration
  scaler = torch.amp.GradScaler(enabled=(device.type == 'cuda'))

  # Set up TensorBoard logging and save model architecture
  writer = SummaryWriter("./"+logs_dir+"/"+experiment_name)
  x = torch.randn(1, input_shape[0], input_shape[1]).to(device)
  writer.add_graph(rnn_model, x)

  # Train model and track training history
  rnn_model, training_history = fit(
      model=rnn_model,
      train_loader=train_loader,
      val_loader=val_loader,
      epochs=epochs,
      criterion=criterion,
      optimizer=optimizer,
      scaler=scaler,
      device=device,
      writer=writer,
      verbose=verbose,
      experiment_name=experiment_name,
      patience=patience,
      learning_rate=learning_rate,
      l1_lambda=l1_lambda,
      l2_lambda=l2_lambda
      )

## Model Training

In [ ]:
# Grid search
rnn_types = ['RNN', 'LSTM']
bidirectionals = [True, False]


for t, b in product(rnn_types, bidirectionals):
  experiment_name = f"rnn_{t}_bidirectional_{b}"
  run_model(rnn_type = t, bidirectional=b, window_size=25, stride=2, batch_size=128, experiment_name=experiment_name, learning_rate=1e-3, epochs=500, patience=50, dropout_rate=0.3, l1_lambda=0, l2_lambda=0, hidden_layers=2, hidden_size=128, criterion=nn.CrossEntropyLoss(weight=weights_tensor, label_smoothing=0.1), verbose=0)

## K-shuffle cross validation

In [ ]:
# Parameters
K = 5
N_VAL_USERS = int(PERCENTAGE_VALIDATION*np.floor(len(df['sample_index'].unique())))          # Number of users for validation split
N_TEST_USERS = 0         # Number of users for test split

# Training
EPOCHS = 500             # Maximum epochs (increase to improve performance)
PATIENCE = 100            # Early stopping patience (increase to improve performance)
VERBOSE = 10             # Print frequency

# Optimisation
LEARNING_RATE = 1e-3     # Learning rate
BATCH_SIZE = 512         # Batch size
WINDOW_SIZE = 24        # Input window size
STRIDE = 5              # Input stride

# Architecture
HIDDEN_LAYERS = 1        # Hidden layers
HIDDEN_SIZE = 128        # Neurons per layer
RNN_TYPE = 'RNN'         # Type of RNN architecture
BIDIRECTIONAL = False    # Bidirectional RNN

# Regularisation
DROPOUT_RATE = 0.5       # Dropout probability
L1_LAMBDA = 0.000001            # L1 penalty
L2_LAMBDA = 0.001           # L2 penalty

# Training utilities
criterion = nn.CrossEntropyLoss(weight=weights_tensor, label_smoothing=0.1)

def k_shuffle_split_cross_validation_round_rnn(df, epochs, criterion, device,
                            k, n_val_users, n_test_users, batch_size, hidden_layers, hidden_size, learning_rate, dropout_rate,
                            window_size, stride, rnn_type, bidirectional,
                            l1_lambda=0, l2_lambda=0, patience=0, evaluation_metric="val_f1", mode='max',
                            restore_best_weights=True, writer=None, verbose=10, seed=42, experiment_name="",split_mins_list=None, split_maxs_list=None):
    """
    Perform K-fold shuffle split cross-validation with user-based splitting for time series data.

    Args:
        df: DataFrame with columns ['user_id', 'activity', 'x_axis', 'y_axis', 'z_axis', 'id']
        epochs: Number of training epochs
        criterion: Loss function
        device: torch.device for computation
        k: Number of cross-validation splits
        n_val_users: Number of users for validation set
        n_test_users: Number of users for test set
        batch_size: Batch size for training
        hidden_layers: Number of recurrent layers
        hidden_size: Hidden state dimensionality
        learning_rate: Learning rate for optimizer
        dropout_rate: Dropout rate
        window_size: Length of sliding windows
        stride: Step size for sliding windows
        rnn_type: Type of RNN ('RNN', 'LSTM', 'GRU')
        bidirectional: Whether to use bidirectional RNN
        l1_lambda: L1 regularization coefficient (if used)
        l2_lambda: L2 regularization coefficient (weight_decay)
        patience: Early stopping patience
        evaluation_metric: Metric to monitor for early stopping
        mode: 'max' or 'min' for evaluation metric
        restore_best_weights: Whether to restore best weights after training
        writer: TensorBoard writer
        verbose: Verbosity level
        seed: Random seed
        experiment_name: Name for experiment logging

    Returns:
        fold_losses: Dict with validation losses for each split
        fold_metrics: Dict with validation F1 scores for each split
        best_scores: Dict with best F1 score for each split plus mean and std
    """

    # Initialise containers for results across all splits
    fold_losses = {}
    fold_metrics = {}
    best_scores = {}

    # Get model architecture parameters
    in_features = len(DATA_COLUMNS)
    num_classes = len(df['label'].unique())

    # Initialise model architecture
    model = RecurrentClassifier(
        input_size=in_features,
        hidden_size=hidden_size,
        num_layers=hidden_layers,
        num_classes=num_classes,
        dropout_rate=dropout_rate,
        bidirectional=bidirectional,
        rnn_type=rnn_type
    ).to(device)

    # Store initial weights to reset model for each split
    initial_state = copy.deepcopy(model.state_dict())

    # Iterate through K random splits
    for split_idx in range(k):

        if verbose > 0:
            print(f"Split {split_idx+1}/{k}")

        # Get unique user IDs and shuffle them with split-specific seed
        unique_users = df['sample_index'].unique()
        random.seed(seed + split_idx)
        random.shuffle(unique_users)

        # Calculate the number of users for the training set
        n_train_users = len(unique_users) - n_val_users - n_test_users

        # Split the shuffled user IDs into training, validation, and test sets
        train_users = unique_users[:n_train_users]
        val_users = unique_users[n_train_users:n_train_users + n_val_users]
        test_users = unique_users[n_train_users + n_val_users:]

        # Split the dataset into training, validation, and test sets based on user IDs
        df_train = df[df['sample_index'].isin(train_users)].copy()
        df_val = df[df['sample_index'].isin(val_users)].copy()
        df_test = df[df['sample_index'].isin(test_users)].copy()

        # Define a mapping of activity names to integer labels
        label_mapping = {
            'no_pain': 0,
            'low_pain': 1,
            'high_pain': 2,
        }

        # Map label names to integers in the training set
        df_train['label'] = df_train['label'].map(label_mapping)

        # Map label names to integers in the validation set
        df_val['label'] = df_val['label'].map(label_mapping)

        # Map label names to integers in the test set
        df_test['label'] = df_test['label'].map(label_mapping)

        if verbose > 0:
            print(f"  Training set shape: {df_train.shape}")
            print(f"  Validation set shape: {df_val.shape}")
            print(f"  Test set shape: {df_test.shape}")

        # Normalise features using training set statistics
        train_max = df_train[DATA_COLUMNS].max()
        train_min = df_train[DATA_COLUMNS].min()

        if split_mins_list is not None and split_maxs_list is not None:
          split_mins_list.append(train_min.copy())
          split_maxs_list.append(train_max.copy())

        df_train[DATA_COLUMNS] = (df_train[DATA_COLUMNS] - train_min) / (train_max - train_min + 1e-8)
        df_val[DATA_COLUMNS] = (df_val[DATA_COLUMNS] - train_min) / (train_max - train_min + 1e-8)
        df_test[DATA_COLUMNS] = (df_test[DATA_COLUMNS] - train_min) / (train_max - train_min + 1e-8)

        # Build sequences using the existing build_sequences function
        X_train, y_train = build_sequences(df_train, window=window_size, stride=stride)
        X_val, y_val = build_sequences(df_val, window=window_size, stride=stride)
        X_test, y_test = build_sequences(df_test, window=window_size, stride=stride)

        if verbose > 0:
            print(f"  Training sequences shape: {X_train.shape}")
            print(f"  Validation sequences shape: {X_val.shape}")
            print(f"  Test sequences shape: {X_test.shape}")

        # Create PyTorch datasets
        train_ds = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
        val_ds   = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))
        test_ds  = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

        # Create data loaders
        train_loader = make_loader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
        val_loader   = make_loader(val_ds, batch_size=batch_size, shuffle=False, drop_last=False)
        test_loader  = make_loader(test_ds, batch_size=batch_size, shuffle=False, drop_last=False)

        # Reset model to initial weights for fair comparison across splits
        model.load_state_dict(initial_state)

        # Define optimizer with L2 regularization
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=l2_lambda)

        # Enable mixed precision training for GPU acceleration
        split_scaler = torch.amp.GradScaler(enabled=(device.type == 'cuda'))

        # Create directory for model checkpoints
        os.makedirs(f"models/{experiment_name}", exist_ok=True)


        writer = SummaryWriter("./"+logs_dir+"/"+experiment_name)

        # (Opzionale) aggiungi il grafo del modello una volta per split.
        # Qui usiamo la shape reale dei dati: (window_size, n_features)
        sample_shape = X_train.shape[1:]        # es. (WINDOW_SIZE, num_features)
        dummy_input = torch.randn(1, *sample_shape).to(device)
        writer.add_graph(model, dummy_input)

        # Train model on current split
        model, training_history = fit(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            epochs=epochs,
            criterion=criterion,
            optimizer=optimizer,
            scaler=split_scaler,
            device=device,
            writer=writer,
            patience=patience,
            verbose=verbose,
            l1_lambda=l1_lambda,
            evaluation_metric=evaluation_metric,
            mode=mode,
            restore_best_weights=restore_best_weights,
            experiment_name=experiment_name+"_split_"+str(split_idx)
        )

        # Store results for this split
        fold_losses[f"split_{split_idx}"] = training_history['val_loss']
        fold_metrics[f"split_{split_idx}"] = training_history['val_f1']
        best_scores[f"split_{split_idx}"] = max(training_history['val_f1'])

    # Compute mean and standard deviation of best scores across splits
    best_scores["mean"] = np.mean([best_scores[k] for k in best_scores.keys() if k.startswith("split_")])
    best_scores["std"] = np.std([best_scores[k] for k in best_scores.keys() if k.startswith("split_")])

    if verbose > 0:
        print(f"Best score: {best_scores['mean']:.4f}±{best_scores['std']:.4f}")

    return fold_losses, fold_metrics, best_scores

def grid_search_cv_rnn(df, param_grid, fixed_params, cv_params, verbose=True):
    """
    Execute grid search with K-shuffle-split cross-validation for RNN models on time series data.

    Args:
        df: DataFrame with columns ['user_id', 'activity', 'x_axis', 'y_axis', 'z_axis', 'id']
        param_grid: Dict of parameters to test, e.g. {'batch_size': [16, 32], 'rnn_type': ['LSTM', 'GRU']}
        fixed_params: Dict of fixed hyperparameters (hidden_size, learning_rate, window_size, stride, etc.)
        cv_params: Dict of CV settings (epochs, k, patience, criterion, scaler, device, etc.)
        verbose: Print progress for each configuration

    Returns:
        results: Dict with scores for each configuration
        best_config: Dict with best hyperparameter combination
        best_score: Best mean F1 score achieved
    """
    # Generate all parameter combinations
    param_names = list(param_grid.keys())
    param_values = list(param_grid.values())
    combinations = list(product(*param_values))

    results = {}
    best_score = -np.inf
    best_config = None

    total = len(combinations)

    for idx, combo in enumerate(combinations, 1):
        # Create current configuration dict
        current_config = dict(zip(param_names, combo))

        config_str = "_".join([f"{k}_{v}" for k, v in current_config.items()])+f"validation_percentage_{PERCENTAGE_VALIDATION}"

        if verbose:
            print(f"\nConfiguration {idx}/{total}:")
            for param, value in current_config.items():
                print(f"  {param}: {value}")

        # Merge current config with fixed parameters
        run_params = {**fixed_params, **current_config}

        # Execute cross-validation
        _, _, fold_scores = k_shuffle_split_cross_validation_round_rnn(
            df=df,
            experiment_name=config_str,
            **run_params,
            **cv_params
        )

        # Store results
        results[config_str] = fold_scores

        # Track best configuration
        if fold_scores["mean"] > best_score:
            best_score = fold_scores["mean"]
            best_config = current_config.copy()
            if verbose:
                print("  NEW BEST SCORE!")

        if verbose:
            print(f"  F1 Score: {fold_scores['mean']:.4f}±{fold_scores['std']:.4f}")

    return results, best_config, best_score


def plot_top_configurations_rnn(results, k_splits, top_n=5, figsize=(14, 7)):
    """
    Visualise top N RNN configurations with boxplots of F1 scores across CV splits.

    Args:
        results: Dict of results from grid_search_cv_rnn
        k_splits: Number of CV splits used
        top_n: Number of top configurations to display
        figsize: Figure size tuple
    """
    # Sort by mean score
    config_scores = {name: data['mean'] for name, data in results.items()}
    sorted_configs = sorted(config_scores.items(), key=lambda x: x[1], reverse=True)

    # Select top N
    top_configs = sorted_configs[:min(top_n, len(sorted_configs))]

    # Prepare boxplot data
    boxplot_data = []
    labels = []

    # Define a dictionary for replacements, ordered to handle prefixes correctly
    replacements = {
        'batch_size_': 'BS=',
        'learning_rate_': '\nLR=',
        'hidden_layers_': '\nHL=',
        'hidden_size_': '\nHS=',
        'dropout_rate_': '\nDR=',
        'window_size_': '\nWS=',
        'stride_': '\nSTR=',
        'rnn_type_': '\nRNN=',
        'bidirectional_': '\nBIDIR=',
        'l1_lambda_': '\nL1=',
        'l2_lambda_': '\nL2='
    }

    # Replacements for separators
    separator_replacements = {
        '_learning_rate_': '\nLR=',
        '_hidden_layers_': '\nHL=',
        '_hidden_size_': '\nHS=',
        '_dropout_rate_': '\nDR=',
        '_window_size_': '\nWS=',
        '_stride_': '\nSTR=',
        '_rnn_type_': '\nRNN=',
        '_bidirectional_': '\nBIDIR=',
        '_l1_lambda_': '\nL1=',
        '_l2_lambda_': '\nL2=',
        '_': ''
    }

    for config_name, mean_score in top_configs:
        # Extract best score from each split (auto-detect number of splits)
        split_scores = []
        for i in range(k_splits):
            if f'split_{i}' in results[config_name]:
                split_scores.append(results[config_name][f'split_{i}'])
        boxplot_data.append(split_scores)

        # Verify we have the expected number of splits
        if len(split_scores) != k_splits:
            print(f"Warning: Config {config_name} has {len(split_scores)} splits, expected {k_splits}")

        # Create readable label using the replacements dictionary
        readable_label = config_name
        for old, new in replacements.items():
            readable_label = readable_label.replace(old, new)

        # Apply separator replacements
        for old, new in separator_replacements.items():
             readable_label = readable_label.replace(old, new)

        labels.append(f"{readable_label}\n(μ={mean_score:.3f})")

    # Create plot
    fig, ax = plt.subplots(figsize=figsize)
    bp = ax.boxplot(boxplot_data, labels=labels, patch_artist=True,
                    showmeans=True, meanline=True)

    # Styling
    for patch in bp['boxes']:
        patch.set_facecolor('lightblue')
        patch.set_alpha(0.7)

    # Highlight best configuration
    ax.get_xticklabels()[0].set_fontweight('bold')

    ax.set_ylabel('F1 Score')
    ax.set_xlabel('Configuration')
    ax.set_title(f'Top {len(top_configs)} RNN Configurations - F1 Score Distribution Across {k_splits} Splits')
    ax.grid(alpha=0.3, axis='y')

    plt.xticks(rotation=0, ha='center')
    plt.tight_layout()
    plt.show()

In [ ]:
%%time
#submissionLSTM_BITrue_HS64_HL1_DR0.3_LR0.001.csv
# Define parameters to search
param_grid = {
    'window_size': [24],
    'stride': [2],
    'rnn_type': ['GRU'],
    'batch_size': [512],
    'bidirectional' : [False]
}

# Fixed hyperparameters (not being tuned)
fixed_params = {
    'learning_rate': 0.0005,
    'hidden_layers': 2,
    'hidden_size': 192,
    'dropout_rate': 0.3,
    'l1_lambda': 0,
    'l2_lambda': 0.0015,
}

split_mins = []
split_maxs = []

# Cross-validation settings
cv_params = {
    'epochs': EPOCHS,
    'criterion': criterion,
    'device': device,
    'k': K,
    'n_val_users': int(N_VAL_USERS),
    'n_test_users': int(N_TEST_USERS),
    'patience': PATIENCE,
    'verbose': 10,
    'seed': SEED,
    'split_mins_list': split_mins,
    'split_maxs_list': split_maxs
}




# Execute search
results, best_config, best_score = grid_search_cv_rnn(
    df=df,
    param_grid=param_grid,
    fixed_params=fixed_params,
    cv_params=cv_params
)


## Ensemble

In [ ]:
MODEL_PATH = []
name=""
for i in range(K):
    MODEL_PATH.append("models/"+name+"_split_"+str(i)+"_model.pth")
BATCH_SIZE_TEST = 512
WINDOW_SIZE_TEST = 24
STRIDE_TEST = 2


# ==============================
# Prepare df_test after preprocessing and before normalization
# ==============================
df_test_raw = df_test_pre_normalization.copy()

# ==============================
# Build K versions of df_test and normalize with the correct split
# ==============================
test_loaders = []
user_ids_ref = None

# split_mins e split_maxs devono essere liste di Series, una per split
assert len(split_mins) == len(MODEL_PATHS) == len(split_maxs), "Mismatch between split_mins/split_maxs and MODEL_PATHS"

for split_idx, (train_min, train_max) in enumerate(zip(split_mins, split_maxs)):
    print(f"Preparing test set for split {split_idx}...")

    # df_test copy
    df_test_i = df_test_raw.copy()

    # normalization for this split
    df_test_i[scale_columns] = (df_test_i[scale_columns] - train_min[scale_columns]) / (
        train_max[scale_columns] - train_min[scale_columns]
    )

    # Build test sequences for this split
    X_test_i, user_ids = build_sequences_test(
        df_test_i,
        window=WINDOW_SIZE_TEST,
        stride=STRIDE_TEST
    )

    # user_id consistency check between splits
    if user_ids_ref is None:
        user_ids_ref = user_ids
    else:
        if not np.array_equal(user_ids_ref, user_ids):
            raise ValueError("user_ids different between split: check build_sequences_test!")

    X_test_i = np.array(X_test_i, dtype=np.float32)

    test_ds_i = TensorDataset(torch.from_numpy(X_test_i))
    test_loader_i = make_loader(
        test_ds_i,
        batch_size=BATCH_SIZE_TEST,
        shuffle=False,
        drop_last=False
    )
    test_loaders.append(test_loader_i)

# Handle empty test sets from user splits
if user_ids_ref is None or len(user_ids_ref) == 0:
    print("WARNING: Test set is empty dopo la costruzione delle sequenze. Skipping.")
else:
    user_ids = user_ids_ref

    # ==============================
    # K models ensemble
    # ==============================
    models = []
    for path in MODEL_PATHS:
        m = torch.load(path, map_location=device, weights_only=False)
        m.to(device)
        m.eval()
        models.append(m)

    all_logits = []

    with torch.no_grad():
        for model_idx, m in enumerate(models):
            print(f"Inferenza modello {model_idx}...")
            model_logits_batches = []

            for xb in test_loaders[model_idx]:
                xb = xb[0].to(device)
                logits = m(xb)
                model_logits_batches.append(logits.cpu().numpy())

            if len(model_logits_batches) == 0:
                continue

            model_logits = np.concatenate(model_logits_batches, axis=0)
            all_logits.append(model_logits)

    if len(all_logits) == 0:
        print("WARNING: Nessun logit prodotto dai modelli. Skipping.")
    else:
        #Logits avarage for K splits
        mean_logits = np.mean(all_logits, axis=0)  # (N_seq, num_classes)
        split_test_preds = mean_logits.argmax(axis=1)

        # ==============================
        # User majority vote
        # ==============================
        final_preds = {}

        for uid in np.unique(user_ids):
            mask = user_ids == uid
            user_preds = split_test_preds[mask]
            majority_class = Counter(user_preds).most_common(1)[0][0]
            final_preds[uid] = majority_class

        #Converts numerical predictions into literal
        label_map = {0: "no_pain", 1: "low_pain", 2: "high_pain"}

        submission_data = []
        for uid, pred_num in final_preds.items():
            label_str = label_map[pred_num]
            submission_data.append((f"{int(uid):03d}", label_str))  # format
